# Send Requests

## Launch a server

This code uses `subprocess.Popen` to start an SGLang server process, equivalent to executing 

```bash
python -m sglang.launch_server --model-path meta-llama/Meta-Llama-3.1-8B-Instruct --port 30000 \
--api-key sk-None-hello-world --host 0.0.0.0 --log-level warning
```
in your command line and wait for the server to be ready.

In [1]:
import subprocess
import sys
import time
import requests

server_process = subprocess.Popen(
    [
        "python",
        "-m",
        "sglang.launch_server",
        "--model-path",
        "meta-llama/Meta-Llama-3.1-8B-Instruct",
        "--port",
        "30000",
        "--api-key",
        "sk-None-hello-world",
        "--host",
        "0.0.0.0",
        "--log-level",
        "error",
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

while True:
    try:
        response = requests.get(
            "http://localhost:30000/v1/models",
            headers={"Authorization": "Bearer sk-None-hello-world"},
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Server is ready. Proceeding with the next steps.")

Server is ready. Proceeding with the next steps.


## Send a Request

Once the server is running, you can send test requests using curl.

In [2]:
!curl http://localhost:30000/v1/chat/completions \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer sk-None-hello-world" \
  -d '{"model": "meta-llama/Meta-Llama-3.1-8B-Instruct", "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "What is a LLM?"}]}'

{"id":"865df79b13d14105b824e580c12e4179","object":"chat.completion","created":1729750400,"model":"meta-llama/Meta-Llama-3.1-8B-Instruct","choices":[{"index":0,"message":{"role":"assistant","content":"LLM stands for Large Language Model. It's a type of artificial intelligence (AI) designed to process and generate human-like language. LLMs are trained on vast amounts of text data, allowing them to learn patterns, relationships, and context in language.\n\nLarge Language Models use complex algorithms and neural networks to analyze and understand the meaning of words, phrases, and sentences. They can then generate text, answer questions, translate languages, and even create original content like stories or conversations.\n\nSome key features of LLMs include:\n\n1. **Natural Language Understanding (NLU)**: LLMs can comprehend language nuances, idioms, and context, making them effective at understanding human language.\n2. **Language Generation**: LLMs can generate text, responses, or even e

## Using OpenAI Compatible API

SGLang supports OpenAI-compatible APIs. Here are Python examples:

In [3]:
import openai

# Always assign an api_key, even if not specified during server initialization.
# Setting an API key during server initialization is strongly recommended.

client = openai.Client(
    base_url="http://127.0.0.1:30000/v1", api_key="sk-None-hello-world"
)

# Chat completion example

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful AI assistant"},
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)
print(response)

ChatCompletion(id='13b434c7cede4baabae4beb529ad2295', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Here are 3 countries and their capitals:\n\n1. **Country:** Japan\n**Capital:** Tokyo\n\n2. **Country:** Australia\n**Capital:** Canberra\n\n3. **Country:** Brazil\n**Capital:** Brasília', refusal=None, role='assistant', function_call=None, tool_calls=None), matched_stop=128009)], created=1729750401, model='meta-llama/Meta-Llama-3.1-8B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=46, prompt_tokens=49, total_tokens=95, prompt_tokens_details=None))


## Use Embedding Models



In [4]:
server_process.terminate()

# Start a new server process for embedding models
# Equivalent to running this in the shell:
# python -m sglang.launch_server --model-path Alibaba-NLP/gte-Qwen2-7B-instruct --port 30010 --api-key sk-None-hello-world --host 0.0.0.0 --is-embedding --log-level error
embedding_process = subprocess.Popen(
    [
        "python",
        "-m",
        "sglang.launch_server",
        "--model-path",
        "Alibaba-NLP/gte-Qwen2-7B-instruct",
        "--port",
        "30010",
        "--api-key",
        "sk-None-hello-world",
        "--host",
        "0.0.0.0",
        "--is-embedding",
        "--log-level",
        "error",
    ],
    text=True,
    stdout=subprocess.DEVNULL,
    stderr=subprocess.DEVNULL,
)

while True:
    try:
        response = requests.get(
            "http://localhost:30010/v1/models",
            headers={"Authorization": "Bearer sk-None-hello-world"},
        )
        if response.status_code == 200:
            break
    except requests.exceptions.RequestException:
        time.sleep(1)

print("Embedding server is ready. Proceeding with the next steps.")

Embedding server is ready. Proceeding with the next steps.


## Use Curl

In [5]:
# Get the first 10 elements of the embedding

! curl -s http://localhost:30010/v1/embeddings \
  -H "Content-Type: application/json" \
  -H "Authorization: Bearer sk-None-hello-world" \
  -d '{"model": "Alibaba-NLP/gte-Qwen2-7B-instruct", "input": "Once upon a time"}' \
  | python3 -c "import sys, json; print(json.load(sys.stdin)['data'][0]['embedding'][:10])"

[0.0083160400390625, 0.0006804466247558594, -0.00809478759765625, -0.0006995201110839844, 0.0143890380859375, -0.0090179443359375, 0.01238250732421875, 0.00209808349609375, 0.0062103271484375, -0.003047943115234375]


## Using OpenAI Compatible API

In [6]:
import openai

client = openai.Client(
    base_url="http://127.0.0.1:30010/v1", api_key="sk-None-hello-world"
)

# Text embedding example
response = client.embeddings.create(
    model="Alibaba-NLP/gte-Qwen2-7B-instruct",
    input="How are you today",
)

embedding = response.data[0].embedding[:10]
print(embedding)

[0.00603485107421875, -0.0190582275390625, -0.01273345947265625, 0.01552581787109375, 0.0066680908203125, -0.0135955810546875, 0.01131439208984375, 0.0013713836669921875, -0.0089874267578125, 0.021759033203125]
